In [13]:
####################################################################
############# All the Dependencies #################################
####################################################################
import pandas as pd
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM,MaxPooling2D,MaxPooling1D,Convolution1D
from keras.layers import Dense,Activation , Dropout,GRU, Bidirectional, Flatten,Conv1D
import plotly.graph_objects as go
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_score,accuracy_score,roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from plotly.offline import plot
import plotly.offline as py

In [14]:
import warnings
warnings.filterwarnings("ignore")

In [16]:
####################################################################
############# Load the data  #######################################
####################################################################
x_train=pd.read_csv('X_train.csv')
y_train=pd.read_csv('y_train.csv').values
x_test=pd.read_csv('X_test.csv')
y_test=pd.read_csv('y_test.csv').values

In [18]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(125973, 123)
(125973, 1)
(22544, 123)
(22544, 1)


In [20]:
####################################################################
############# Selecting best 16 features ###########################
####################################################################
x_train=x_train[['dst_host_srv_count','dst_host_same_srv_rate','dst_host_serror_rate','src_bytes','dst_host_diff_srv_rate','count','logged_in','dst_host_srv_diff_host_rate','diffic','dst_host_same_src_port_rate','dst_host_count','duration','root_shell','service_http','flag_SF','dst_host_rerror_rate']]
x_test=x_test[['dst_host_srv_count','dst_host_same_srv_rate','dst_host_serror_rate','src_bytes','dst_host_diff_srv_rate','count','logged_in','dst_host_srv_diff_host_rate','diffic','dst_host_same_src_port_rate','dst_host_count','duration','root_shell','service_http','flag_SF','dst_host_rerror_rate']]

In [21]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(125973, 16)
(125973, 1)
(22544, 16)
(22544, 1)


In [12]:
#df to numpy array
x_train=x_train.values
x_test=x_test.values

In [6]:
####################################################################
############# Combined test and train data #########################
####################################################################
x_data=np.concatenate([x_train,x_test],axis=0)
y_data=np.concatenate([y_train,y_test],axis=0)

In [7]:
####################################################################
############# Train Test Split #####################################
####################################################################
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=42)

In [8]:
####################################################################
############# Target Data Conversion into categorical ##############
####################################################################
y_test=to_categorical(y_test)
y_train=to_categorical(y_train)
tru=[list(i).index(max(i)) for i in y_test]

In [6]:
####################################################################
############# Scaling Data #########################################
####################################################################
scalar=MinMaxScaler()
x_train=scalar.fit_transform(x_train)
x_test=scalar.fit_transform(x_test)

In [15]:
####################################################################
############# Training Hybrid Model ################################
####################################################################
hybrid = Sequential()
hybrid.add(Convolution1D(48, 3, padding="same",activation="relu",input_shape=(16, 1)))
hybrid.add(Convolution1D(48, 3, padding="same", activation="relu"))
hybrid.add(MaxPooling1D(pool_size=(2)))
hybrid.add(Convolution1D(48, 3, padding="same", activation="relu"))
hybrid.add(Convolution1D(48, 3, padding="same", activation="relu"))
hybrid.add(MaxPooling1D(pool_size=(2)))
hybrid.add(LSTM(100))
hybrid.add(Dropout(0.1))
hybrid.add(Dense(2, activation="softmax"))
hybrid.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])
hybrid.fit(x_train.reshape(len(x_train), len(x_train[0]),1), y_train,epochs=10,verbose=1,batch_size = 500)
hybrid.save('hybrid.h5')

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential_3/conv1d_6/conv1d (defined at <ipython-input-15-8cafab53471a>:15) ]] [Op:__inference_train_function_4323]

Function call stack:
train_function


In [17]:
####################################################################
############# Training GRU Model ################################
####################################################################
gru = Sequential()
gru.add(GRU(100, activation='relu', return_sequences=True, input_dim = 16))
gru.add(GRU(100, activation='relu'))
gru.add(Dense(2, activation = 'softmax'))
gru.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
gru.fit(x_train.reshape(len(x_train),1, len(x_train[0])), y_train,epochs=10,verbose=1,batch_size = 500)
gru.save('gru.h5')

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


InternalError:  Blas GEMM launch failed : a.shape=(500, 100), b.shape=(100, 300), m=500, n=300, k=100
	 [[{{node sequential_5/gru_2/while/body/_1/sequential_5/gru_2/while/gru_cell_2/MatMul_1}}]] [Op:__inference_train_function_8472]

Function call stack:
train_function


In [22]:
# load and evaluate a saved model
from numpy import loadtxt
from keras.models import load_model
# @tf.autograph.experimental.do_not_convert
import os
import tensorflow as tf
tf.autograph.set_verbosity(0)
# load model
# model = load_model('gru.h5')

# It can be used to reconstruct the model identically.
reconstructed_model = load_model("gru.h5")
reconstructed_model.fit(x_test, y_test)
# np.testing.assert_allclose(
#     model.predict(x_test), reconstructed_model.predict(x_test)
# )
# summarize model.
# model.summary()
# load dataset
# dataset = loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
# X = dataset[:,0:8]
# Y = dataset[:,8]
# evaluate the model
# score = model.evaluate(x_test, y_test, verbose=0)
# print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


ValueError: Input 0 of layer sequential_2 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: [None, 16]

In [22]:
x_test.shape

(44556, 16)

In [24]:
y_test.shape

(44556, 2)

In [26]:
y_test

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [28]:
x_train.shape

(103961, 16)

In [29]:
y_train.shape

(103961, 2)